In [ ]:
from google.colab import drive
import random
from platform import python_version
import pandas as pd
from sklearn.feature_selection import f_classif
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, cohen_kappa_score, f1_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
random.seed(0)
print(python_version())


3.10.11


In [ ]:
drive.mount('/content/drive') # ignore this part if you are running notebook file in your local system

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''import os
os.chdir('/content/drive/Shared with me/AI-Project')'''

"import os\nos.chdir('/content/drive/Shared with me/AI-Project')"

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data.csv") # needs to be replaced by the correct file path in your system

In [ ]:
'''df = df.iloc[: , 1:]  #deleting 1st column 
df = df.drop(['Donor_Sample'], axis=1) #deleting this column
df = df.fillna(0)'''

"df = df.iloc[: , 1:]  #deleting 1st column \ndf = df.drop(['Donor_Sample'], axis=1) #deleting this column\ndf = df.fillna(0)"

In [ ]:
#df.head(5)

In [ ]:
df = df.iloc[: , 1:]  #deleting 1st column 
df = df.drop(['Donor_Sample'], axis=1) #deleting this column
df = df.fillna(0)

In [ ]:
#target = df['is_tumor']
df.is_tumor = df.is_tumor.astype(int) #converted is_tumor value float to int

In [ ]:
print("Number of markers/ features", len(df.columns)-1)
print("Number of samples", len(df))
print(df['is_tumor'].value_counts())
#print(df)

Number of markers/ features 24981
Number of samples 337
1    309
0     28
Name: is_tumor, dtype: int64


In [ ]:
#split dataset into features and target
X = df.iloc[:,0:len(df.columns)-1].values
print(type(X))
print("X = ", X.shape)
Y = df.iloc[:,len(df.columns)-1].values
print("Y = ",Y.shape)

<class 'numpy.ndarray'>
X =  (337, 24981)
Y =  (337,)


In [ ]:

sc = StandardScaler()
X = sc.fit_transform(X)
#print(X)
print(X.shape)
print(Y.shape)
Y_original = Y

(337, 24981)
(337,)


In [ ]:

oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)
print(X.shape)
print(Y.shape)
#print(target.shape)

(618, 24981)
(618,)


In [ ]:
target = pd.DataFrame(Y) 

In [ ]:
print("train and test split on total dataset", len(X))
X_train0, X_test, y_train0, y_test = train_test_split(X, Y, test_size=0.2, random_state = 0)
print("train and validation split on training dataset", len(X_train0))
X_train, X_val, y_train, y_val = train_test_split(X_train0, y_train0, test_size=0.2, random_state = 0)
print("Training data:",X_train.shape)
print("Validation data", X_val.shape)
print("test data",X_test.shape)

train and test split on total dataset 618
train and validation split on training dataset 494
Training data: (395, 24981)
Validation data (99, 24981)
test data (124, 24981)


In [ ]:
#Only Anova
%%time
print("Total number of features:", X.shape[1])
#a = round(0.85*(X.shape[1]))
#print("85% of total feature", a)
X_new = SelectKBest(f_classif, k=X.shape[1]).fit_transform(X, Y)
#df=pd.DataFrame(X_new)
fvals, pvals = f_classif(X_new, Y)
to_remove = pvals >= (0.05/X_new.shape[1])
X_anova = np.delete(X_new, obj=to_remove, axis=1)
print("Number of selected features using ANOVA F-test:", X_anova.shape[1])

X_anova = pd.DataFrame(X_anova)
X_anova['is_tumor'] = target
#X_anova.head()
#pd.DataFrame(X_anova).to_csv("/content/drive/MyDrive/only_anova_markers.csv") # needs to be replaced by the correct file path in your system


Total number of features: 24981
Number of selected features using ANOVA F-test: 15340
CPU times: user 711 ms, sys: 460 ms, total: 1.17 s
Wall time: 1.77 s


In [ ]:
#Only Random forest
%%time
print("Only random forest")
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)
rf_sel_features=sel.get_support()
X_rf = np.delete(X, obj=~np.array(rf_sel_features), axis=1)
print("Number of selected features using RF:", X_rf.shape[1])

X_rf = pd.DataFrame(X_rf)
X_rf['is_tumor'] = target

#pd.DataFrame(X_rf).to_csv("/content/drive/MyDrive/only_rf_markers.csv") # needs to be replaced by the correct file path in your system

Only random forest
Number of selected features using RF: 538
CPU times: user 2.23 s, sys: 27.9 ms, total: 2.26 s
Wall time: 5.34 s


In [ ]:
# Use LinearRegression
sel_LR = SelectFromModel(LinearRegression())
sel_LR.fit(X, Y)
# Get selected features
sel_LR_features = sel_LR.get_support()

X_LinearRegression = np.delete(X, obj=sel_LR_features, axis=1)

print('Reduced number of features (Linear Regression):', X_LinearRegression.shape[1])

X_LinearRegression = pd.DataFrame(X_LinearRegression)
X_LinearRegression['is_tumor'] = target

#pd.DataFrame(X_LinearRegression).to_csv("/content/drive/MyDrive/only_LR_markers.csv") # needs to be replaced by the correct file path in your system

Reduced number of features (Linear Regression): 15455


In [ ]:
#
# Use LogisticRegression
sel_LgR = SelectFromModel(LogisticRegression(random_state=10, max_iter=1000))
sel_LgR.fit(X, Y)
# Get selected features
sel_LgR_features = sel_LgR.get_support()

X_LogisticRegression = np.delete(X, obj=sel_LgR_features, axis=1)

print('Reduced number of features (Logistic Regression):', X_LogisticRegression.shape[1])

X_LogisticRegression = pd.DataFrame(X_LogisticRegression)
X_LogisticRegression['is_tumor'] = target

#pd.DataFrame(X_LogisticRegression).to_csv("/content/drive/MyDrive/only_LgR_markers.csv") # needs to be replaced by the correct file path in your system

Reduced number of features (Logistic Regression): 15313


In [ ]:
'''# Use PCA
sel_pca = SelectFromModel(PCA()).fit(X)
sel_pca.transform(X)
# Get selected features
sel_pca_features = sel_pca.get_support()
X_pca = np.delete(X, obj=sel_pca_features, axis=1)
'''

#print('Reduced number of features (PCA):', X_pca.shape[1])

'# Use PCA\nsel_pca = SelectFromModel(PCA()).fit(X)\nsel_pca.transform(X)\n# Get selected features\nsel_pca_features = sel_pca.get_support()\nX_pca = np.delete(X, obj=sel_pca_features, axis=1)\n'

In [ ]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, Y)
model = SelectFromModel(lsvc, prefit=True)
X_svc = model.transform(X)
print("Number of selected features using linear SVC:", X_svc.shape[1])

X_svc = pd.DataFrame(X_svc)
X_svc['is_tumor'] = target

pd.DataFrame(X_svc).to_csv("/content/drive/MyDrive/only_SVC_markers.csv") # needs to be replaced by the correct file path in your system

Number of selected features using linear SVC: 26


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)

'''sel = SelectFromModel(pca)
sel.fit(X_train, y_train)
rf_sel_features=sel.get_support()
X_pca = np.delete(X, obj=~np.array(rf_sel_features), axis=1)'''

print("Number of selected features using RF:", X_pca)

Number of selected features using RF: [[ 7.33762694e+01 -5.47422518e+00]
 [-3.65617818e+00 -5.77276382e-02]
 [-5.89971048e+01  1.66544916e+00]
 ...
 [-6.05189300e+01  4.65587796e+00]
 [-4.36001323e+01  9.99150854e+00]
 [-5.52082029e+01 -1.73817003e+01]]


In [ ]:
'''print("\n")
print("Both ANOVA and random forest")
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_anova, Y)
anova_rf_sel_features=sel.get_support()
X_anova_rf = np.delete(X_anova, obj=~np.array(anova_rf_sel_features), axis=1)
print("Number of selected features using anova and RF:", X_anova_rf.shape[1])'''

'print("\n")\nprint("Both ANOVA and random forest")\nsel = SelectFromModel(RandomForestClassifier(n_estimators = 100))\nsel.fit(X_anova, Y)\nanova_rf_sel_features=sel.get_support()\nX_anova_rf = np.delete(X_anova, obj=~np.array(anova_rf_sel_features), axis=1)\nprint("Number of selected features using anova and RF:", X_anova_rf.shape[1])'

In [ ]:
# Use LinearRegression
sel_LR = SelectFromModel(LinearRegression())
sel_LR.fit(X, Y)
# Get selected features
sel_LR_features = sel_LR.get_support()

X_LinearRegression = np.delete(X, obj=sel_LR_features, axis=1)

print('Reduced number of features (Linear Regression):', X_LinearRegression.shape[1])


# Use LogisticRegression
sel_LgR = SelectFromModel(LogisticRegression(random_state=10, max_iter=1000))
sel_LgR.fit(X_LinearRegression, Y)
# Get selected features
sel_LgR_features = sel_LgR.get_support()

X_LogisticRegression = np.delete(X_LinearRegression, obj=sel_LgR_features, axis=1)

print('Reduced number of features (linear & Logistic Regression):', X_LogisticRegression.shape[1])

#Only Anova
#print("Total number of features:", X.shape[1])
#a = round(0.85*(X.shape[1]))
#print("85% of total feature", a)
X_new = SelectKBest(f_classif, k=X_LogisticRegression.shape[1]).fit_transform(X_LogisticRegression, Y)
#df=pd.DataFrame(X_new)
fvals, pvals = f_classif(X_new, Y)
to_remove = pvals >= (0.05/X_new.shape[1])
X_anova = np.delete(X_new, obj=to_remove, axis=1)
print("Number of selected features using linear & Logistic Regression & ANOVA:", X_anova.shape[1])

#Only Random forest
print("Only random forest")
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_anova, Y)
rf_sel_features=sel.get_support()
X_rf = np.delete(X_anova, obj=~np.array(rf_sel_features), axis=1)
print("Number of selected features using linear & Logistic Regression, ANOVA & Random forest:", X_rf.shape[1])


lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_rf, Y)
model = SelectFromModel(lsvc, prefit=True)
X_svc = model.transform(X_rf)
print("Number of selected features using linear SVC:", X_svc.shape[1])

X_svc = pd.DataFrame(X_svc)
X_svc['is_tumor'] = target

pd.DataFrame(X_svc).to_csv("/content/drive/MyDrive/LR_LgR_Anova_RF_lsvc_markers.csv") # needs to be replaced by the correct file path in your system

Reduced number of features (Linear Regression): 15455
Reduced number of features (linear & Logistic Regression): 9218
Number of selected features using linear & Logistic Regression & ANOVA: 4602
Only random forest
Number of selected features using linear & Logistic Regression, ANOVA & Random forest: 431
Number of selected features using linear SVC: 23
